![title](img/logo_white_full.png)

# Dashboards Tutorial

In this part of the tutorial, we are going to learn Plotly and Dash syntax. This will form a basis for you to build your own interactive plots and beautiful dashboards!

**What is Plotly?** 

[Plotly](https://plot.ly/) is a framework that includes visualization tools and Python API library to intuitively build interactive plots. 

**What is Dash?** 

Actually, [Dash](https://plot.ly/products/dash/) is a product of Plotly, and comes with all components required to build modern UX application (sliders, check-boxes, buttons, etc.) with HTML objects to create a graphical layout of a dashboard. Dash is a Python alternative of R Shiny, but the advantage here is that you can directly embed your Plotly graphs in Dash app!

---
## Plotly basics
You can use Plotly in two modes:
* **online** - you need to set up an account in Plotly Chart Studio and created graphs are automatically sent to the web-server.
* **offline** - this is an option we are going to use. Just like matplotlib, it creates the plots locally.

Now, in ```plotly.offline``` mode you can create plots in two ways:
* **plot** - if you create a Plotly chart with ```plot``` method, it is going to be saved in .html file and opened in your local web browser.
* **iplot** - if you create a Plotly chart with ```iplot``` method, it is going to be directly showed in-line iPython notebook. 

Let's create a simple chart in-line Jupyter console.

In [1]:
from plotly.offline import iplot, init_notebook_mode # Interactive graphs
import plotly.graph_objs as go 

init_notebook_mode(connected=True) # Required for Jupyter to produce in-line Plotly graphs

trace0 = dict(
    x=[1, 2, 3, 4],
    y=[10, 15, 13, 17],
    type='scatter'
)
trace1 = dict(
    x=[1, 2, 3, 4],
    y=[16, 5, 11, 9],
    type='scatter'
)

layout = dict(title='Plotly Graph')

fig = go.Figure(data=[trace0, trace1], layout=layout)
iplot(fig)

The ```iplot``` method generates ```go.Figure``` with the following arguments:
* **data**: can have multiple traces in a list. A trace is the source of data for the part of the chart and contains individual settings such as colors, markers size, line width, etc. All information is passed as dictionaries.
* **layout**: this is a layout of the chart i.e. the settings of axes, tick marks, grids, titles, legends, etc. All information is passed as dictionaries.

That is basically the whole syntax of Plotly graphs. All other charts such as 3D surface plots, maps, etc. differ only by the type of chart and arguments inside traces.

These are the chart you are able to do in Plotly:
![Plotly charts](img/plotly_charts.png)

For other examples and full reference, please visit [Plotly docs](https://plot.ly/python/#basic-charts).

## Dash basics
Dash applications are easy to create. Most of the time you will spend on customizing the layout to meet our client's expectations and deploying it on a stable server. 

Let's say you want to create a basic dashboard with two plots side-by-side and one dropdown to select a color of the first plot. It is a dummy example but will serve us as a building block for more advanced apps.

Firstly, we import all the components required. Note two Plotly chart definitions imported from utils.py.

In [9]:
import flask # Flask server on which Dash app is build

import pandas as pd # Data-Frames

import dash # Dash app
from dash.dependencies import Input, Output # For Dash callbacks
import dash_core_components as dcc # Dash components
import dash_html_components as html # Dash html components

from utils_dash import dash_surface, dash_choropleth_map # Custom Utilities written for this tutorial

We initilize a Dash app with Flask server instance and CSS stylesheet that will help us with page layout. The knowledge about how CSS works is not really required, but can turn out very helpful in layout customization. 

We also download yield curve data that will be used for plotting.

In [5]:
df = pd.read_csv('https://raw.githubusercontent.com/quanteeai/dash-yield-curves-demo/master/data/yc.csv')

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']   
server = flask.Flask('app')
app = dash.Dash('app', server=server, external_stylesheets=external_stylesheets)

Dash apps consist of two parts:
* **layout** - definition of how dashboard layout should look like
* **callbacks** - used to define interactions between layout elements (dropdowns, graphs, etc) and User

Let's define above-mentioned layout. It should look like this:

| title    	|        	|
|----------	|--------	|
| dropdown 	|        	|
| graph1   	| graph2 	|

In [6]:
app.layout = html.Div([
    
    # Container for first row: header
    html.H1('Example Dash App'),
    
    # Container for second row: dropdowns
    html.Div([
        
        # Container for dropdown
        html.Div([
            dcc.Dropdown(
                id='main-dropdown',
                options=[
                    {'label': '2019-02-28', 'value': '2019-02-28'},
                    {'label': '2019-01-31', 'value': '2019-01-31'},
                    {'label': '2018-11-30', 'value': '2018-11-30'}
                ],
                value='2019-02-28'
            ), 
        ], className='six columns')
        
    ], className='row'),
    
    # Container for third row: Plotly charts
    html.Div([
        
        # Container for left chart (updated in callback)
        html.Div([
            dcc.Graph(id='left-graph')
        ], className='six columns'),
        
        # Container for right chart (updated straightaway)
        html.Div([
            dcc.Graph(id='right-graph', figure=dash_surface(df))
        ], className='six columns')
        
    ], className='row')
    
])

Note that the layout of the page is divided into multiple ```html.div```. The CSS stylesheet provides us a classNames for these divs (```row```, ```six columns```) so that everything is in the right place. The interactive components such as ```dcc.Dropdown``` and ```dcc.Graph``` has an ```id``` argument which will be used as an identifier in callbacks.

In [7]:
@app.callback(Output('left-graph', 'figure'),
              [Input('main-dropdown', 'value')])
def update_left_graph(selected_dropdown_value):
    dff = df[(df.date == selected_dropdown_value) & (df.term == 1)]
    return dash_choropleth_map(dff)

We define a callback with decorator ```app.callback```. Then, we give one ```Output``` object and one or multiple ```Input``` objects. The function ```update_left_graph``` will be fired on app initilization and then every time the ```Input``` will change its value. Make sense, right?

Now, it's time to run the app. Execute the below code snippet and go to http://127.0.0.1:8050/ to see your app!

In [ ]:
if __name__ == '__main__':
    app.run_server()

To see the possibilities of Dash visit the [deployed demo](https://quantee-dash-yield-curves-demo.herokuapp.com/) that we show to our clients and official [Dash examples](https://dash.plot.ly/gallery). 

**Now, it is time to learn how to deploy our simple app in the next tutorial!**